In [3]:
import pandas as pd
from sqlalchemy.testing.suite.test_reflection import metadata

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

# 1. LLM 기반 질의 생성

이번 실습에서는 총 30개의 QA 데이터 셋을 만들어보겠습니다.

먼저 LLM 기반 질의 생성을 통해 15개의 QA 데이터 셋을 만들어보겠습니다.

15 + 10 + 5 = 30

# 2. Human in the Loop 실습

10개의 QA 데이터 셋을 만들어보겠습니다.

# 3. 이미 FaQ가 있는 경우
5개의 FaQ를 이용해서 만들어보겠습니다 !

qid, query, generation_gt, retrieval_gt를 만들어서 넣어줘야 합니다

FaQ가 있으니, query와 generation_gt는 이미 있습니다.
qid와 retrieval_gt는 직접 만들어주셔야 합니다.

In [18]:
question = [
    "",
    "",
    "",
    "",
    "",
]

In [23]:
answer = [
    "a",
    "b",
    "c",
    "d",
    "e",
]

In [24]:
from uuid import uuid4

qid = [str(uuid4()) for _ in range(len(question))]

In [25]:
generation_gt = [[a] for a in answer]

In [26]:
generation_gt

[['a'], ['b'], ['c'], ['d'], ['e']]

이제 retrieval_gt 만 남았습니다

retrieval_gt를 직접 만드는 방법에는 크게 3가지가 있습니다.

1. 직접 찾기
2. 검색 시스템을 만들어 조금 쉽게 찾아보기

저희는 2번 방법을 사용해 만들어보겠습니다!

### BM25를 이용해 retrieval_gt 매핑 검색 시스템 만들기

In [27]:
corpus_dir = os.path.join(root_dir, 'autorag_project', 'final', 'chunk', '0.parquet')
corpus = pd.read_parquet(corpus_dir)
corpus

,doc_id,contents,path,start_end_idx,metadata
0,c7036095-887a-4082-885e-14209c68cb0f,<h1 id='0' style='font-size:18px'>2024년 KBO 규약...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 575]","{'last_modified_datetime': '2025-01-27', 'next..."
1,fc96bdca-c580-4f27-ab43-dfbcdbda99fb,고용할 수 있음</td></tr><tr><td>제44조 [선수계약의 승인]</td>...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[546, 1071]","{'last_modified_datetime': '2025-01-27', 'next..."
2,18e02344-dc14-4924-b38c-9573bd437fcb,<h1 id='0' style='font-size:20px'>제4장 연 고 권</h...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 619]","{'last_modified_datetime': '2025-01-27', 'next..."
3,fd7ba604-3428-4b9b-8dea-8565b394cb34,히어로즈</td><td>서울특별시</td></tr><tr><td>NC 다이노스</t...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[621, 1277]","{'last_modified_datetime': '2025-01-27', 'next..."
4,ef1e83f5-7a4d-419b-b469-f4c4eb53b101,<table id='0' style='font-size:14px'><tr><td>구...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 516]","{'last_modified_datetime': '2025-01-27', 'next..."
...,...,...,...,...,...
856,bb36c5db-a016-404f-ad42-2e2c1eab11e4,그림 8. 정식경기 \r\n<예1>\r\n이닝 1 2 3 4 5 6 7 8 9 \r...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 438]","{'last_modified_datetime': '2025-01-27', 'next..."
857,167952f2-5c16-461c-b139-f7b7d8cf9f40,갑 1 0 0 1 0 0 1 |3\r\n을 0 0 1 0 0 0 0+∝ |1,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[416, 455]","{'last_modified_datetime': '2025-01-27', 'next..."
858,621dcf2e-3fdd-4b8e-8514-b94353524da3,그림 9. 정식경기\r\n<예8>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 447]","{'last_modified_datetime': '2025-01-27', 'next..."
859,006c3812-75b3-4d24-a703-fa261cd59417,3-2 승 \r\n<예14>\r\n이닝 1 2 3 4 5 6 7 8 9 \r\n갑 ...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[429, 526]","{'last_modified_datetime': '2025-01-27', 'next..."


In [33]:
sample_texts = corpus['contents'].tolist()
doc_ids = corpus['doc_id'].tolist()

In [37]:
from langchain_core.documents import Document

docs = [Document(page_content=sample_text, metadata={"doc_id": id}) for sample_text, id in zip(sample_texts, doc_ids)]

In [36]:
# 테디노트님의 라이브러리 활용
# https://wikidocs.net/251980
from langchain_teddynote.retrievers import KiwiBM25Retriever

kiwi = KiwiBM25Retriever.from_documents(docs)

In [43]:
def pretty_print(docs):
    for doc in docs:
        print(f"id: {doc.metadata['doc_id']}\n")
        print(f"content: {doc.page_content}\n\n")

In [44]:
pretty_print(kiwi.invoke(question[0]))

id: e6e6332e-a897-4c8a-973c-a0be0753bdfd

content: 그림 10. 투구자세 
(본문 5.07(a) 참조) 
와인드업 포지션(오른손 투수) 
세트 포지션(오른손 투수)


id: 9e20ffb4-338b-4986-b24d-1a85e894d9ae

content: 공을 타자에게 투구하는데 실패
한 경우 보크가 선언된다(주자가 없는 경우 볼 선언).
b. 타자 : 자동 스트라이크
타자는 타석에서 과도한 이점을 얻을 목적으로 타석 
입장을 고의로 지체해서는 안 된다. 타자가 피치클
락의 잔여시간이 8초가 남은 시점까지 타석에 입장
해 타격준비자세를 취하지 않으면 즉시 볼 데드 상
황이 되고, 타자는 피치클락 규정 위반으로 심판에 
의해 곧바로 스트라이크가 선언된다. 이 때 어떠한 
주자도 진루할 수 없다.
[주] 타자의 행동으로 투수의 투구 동작이 방해받았다고 심
판이 판단한 경우 타자는 타임을 요청했거나 타석 밖으
로 이탈한 것으로 간주된다. 만약 타자가 동일 타석에서


id: 8301be23-4aa4-49dc-9724-56ca8f35e76e

content: 교대 또는 투수교체 후 첫 번째 투구의 경우 적절한 광
고 송출이 완료되는 것을 보장하기 위해, 투수는 이닝 교대 
또는 투수교체 종료 시점까지 잔여시간이 18초보다 많이 남
았을 때는 투구를 위한 동작을 시작해서는 안 된다.
[주] 타자가 타석에 위치해 타격준비자세를 취하고 있고, 잔여
시간이 18초 이하인 경우 광고 송출이 완료되지 않았다
고 해서 투수가 첫 번째 투구 동작에 들어가는 것을 지연
시킬 수 없다. 단, 본 규정에 의거하여 피치클락의 시간이 
재설정된 경우 타자가 타석에 입장해 타격준비자세를 취
하자마자 첫 번째 투구 동작을 시작할 수 있다.
c. 계측 종료 : 투구 동작이 시작되었을 때 종료
투수가 타자에게 투구하는 자연스러운 동작을 
시작할 때 계측이 종료된다(빈 화면 송출).


id: f2048282-5641-4876-b4de-75be6e3bfa2

In [ ]:
retrieval_gt = [
    [[""]], # 1번째 질문
    [[""]], # 2번째 질문
    [[""]], # 3번째 질문
    [[""]], # 4번째 질문
    [[""]], # 5번째 질문
]

In [ ]:
faq_qa = pd.DataFrame({
    "qid": qid,
    "query": question,
    "generation_gt": generation_gt,
    "retrieval_gt": retrieval_gt,
})

# QA 데이터 셋으로 합치기